In [33]:
import pandas as pd
from pathlib import Path
from datetime import datetime

In [54]:
path = Path("G:\\Otros ordenadores\\Mi PC\\IGAC\\PendienteSubirDataLake")
archivos = [p for p in path.iterdir() if p.is_file() is True]

prueba = archivos[0]

In [12]:
prueba

WindowsPath('G:/Otros ordenadores/Mi PC/IGAC/PendienteSubirDataLake/EXT LEICA_LINEA 2_BUENAVENTURA_ALTODEROSAS.xlsx')

In [70]:
output = []
contador = 0   # si quiere alternar sentido globalmente
for archivo in archivos:
    try:
        df = pd.read_excel(archivo)

        # convertir fechas una sola vez
        fechas = pd.to_datetime(
            df.iloc[:, 4],
            format='%Y-%m-%d',
            errors='coerce'
        )

        for i, fila in enumerate(df.itertuples(index=False)):
            fecha = fechas.iloc[i]

            registro = {
                'ini': fila[0],
                'fin': fila[2],
                'sentido': 'I' if contador % 2 == 0 else 'R',
                'dif_alt': round(fila[12], 6)/10000,
                'dist': round(fila[14], 6)/100,
                'fecha': (
                    f"{fecha.year}-{fecha.month}-{fecha.day}"
                    if pd.notna(fecha)
                    else None
                ),
                'archivo': str(archivo),
                'observaciones': str(fila[16])
            }

            output.append(registro)
            contador += 1
    except:
        continue

In [71]:
import json 
with open("salida.json", "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=4)

In [73]:
import psycopg2
from psycopg2.extras import execute_values

conn = psycopg2.connect(
    dbname="IGAC",
    user="samuel",
    password="leonleo7",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

In [74]:
sql = """
INSERT INTO geodesia.crudos_historicos
(ini, fin, sentido, dif_alt, dist, fecha, archivo, observaciones)
VALUES %s
"""

In [81]:
datos = [
    (
        r["ini"],
        r["fin"],
        r["sentido"],
        r["dif_alt"],
        r["dist"],
        r["fecha"],          # 'YYYY-M-D' o datetime.date
        r["archivo"],
        r.get("observaciones", None)
    )
    for r in output
]

try:
    execute_values(cursor, sql, datos)
    conn.commit()
except Exception as e:
    conn.rollback()   # ← CLAVE
    print("Error real:", e)

In [82]:
len(output)

12012